## Cross-Bucket Tests on all 3 Fragment Types using Kalman Filter

### Part A: (1) Import Fragment Data from MATLAB, (2) Save Them in a .pickle file, and (3) Load Them In

###### (HOWEVER, YOU DON'T NEED TO LOAD THEM IN AGAIN SINCE STEP 1 ALREADY DOES THAT FOR YOU!)

In [1]:
## (Configuration) Allows you to return multiple variables from a single cell ##
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Allows you to import files from another folder in current directory ## 
import os 
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Import standard package ###
import numpy as np
from scipy import io
import sys

# Specify Fragment Types to be used for this Anaylsis 
frag_type = ['AD', 'HV', 'VM'] # ['AccDec', 'HillValley', 'VelMin']

In [20]:
 ## (OLD) Import Data ## 
# We'll load in position data and derive velocity and acceleration from it 

folder = '/Users/rbhatt6/Documents/MATLAB/' # For Windows: folder='C:\\Users\\rbhatt1\\Downloads\\' 

 #locals()["sortIn"+frag_type[i]] = io.loadmat(folder+'cleanedSortIn.mat')
for i in range(len(frag_type)):
    input = "sortIn_"+frag_type[i]
    locals()[input] = io.loadmat(folder + input + '.mat')
   
    outputX = "sortX_"+frag_type[i]
    locals()[outputX] = io.loadmat(folder + outputX + '.mat')

    outputY = "sortY_"+frag_type[i]
    locals()[outputY] = io.loadmat(folder + outputY + '.mat')

    locals()[input] = np.squeeze(list(locals()[input].values())[3])
    locals()[outputX] = np.squeeze(list(locals()[outputX].values())[3])
    locals()[outputY] = np.squeeze(list(locals()[outputY].values())[3])

In [15]:
## (OLD) Save Data in .pickle files ## 
import pickle

data_folder='/Users/rbhatt6/Documents/MATLAB/' # Folder where you want to save the data

for i in range(len(frag_type)):
    input = "sortIn_"+frag_type[i]
    outputX = "sortX_"+frag_type[i]
    outputY = "sortY_"+frag_type[i]

    with open(data_folder + input + '.pickle','wb') as f:
        pickle.dump(locals()[input],f)

    with open(data_folder + outputX + '.pickle','wb') as f:
        pickle.dump(locals()[outputX],f)

    with open(data_folder + outputY + '.pickle','wb') as f:
        pickle.dump(locals()[outputY],f)

In [142]:
## (UPDATED) Import Data ##
# We'll load in position data and derive velocity and acceleration from it 

folder = '/Users/rbhatt6/Documents/MATLAB/' # For Windows: folder='C:\\Users\\rbhatt1\\Downloads\\' 

 #locals()["sortIn"+frag_type[i]] = io.loadmat(folder+'cleanedSortIn.mat')
for i in range(len(frag_type)):
    input = "sortIn_"+frag_type[i]
    locals()[input] = io.loadmat(folder + input + '.mat')

    output = "sortOut_" + frag_type[i]
    locals()[output] = io.loadmat(folder + output + '.mat')

    locals()[input] = np.squeeze(list(locals()[input].values())[3])
    locals()[output] = np.squeeze(list(locals()[output].values())[3])

In [170]:
# dims = 
sortOut_AD.shape
sortOut_AD[0].shape # First row of cells (i.e. kinematics for the first bucket)
sortOut_AD[0][0].shape
sortOut_AD[0][0][0].shape

sortOut_AD[0]

(16, 6)

(6,)

(1633, 1)

(1,)

array([array([[0.00351772],
              [0.0219465 ],
              [0.051092  ],
              ...,
              [0.3131765 ],
              [0.31382569],
              [0.26383715]]), array([[-0.01719953],
                                     [-0.00574728],
                                     [ 0.02476847],
                                     ...,
                                     [ 0.10968373],
                                     [-0.17419172],
                                     [-0.40420583]]),
       array([[-0.00024946],
              [ 0.00139547],
              [ 0.00064977],
              ...,
              [ 0.00474592],
              [-0.00296012],
              [-0.00051165]]), array([[-0.00026559],
                                      [ 0.00110366],
                                      [ 0.00104829],
                                      ...,
                                      [-0.00798282],
                                      [-0.01222617],
             

In [147]:


for i in range(len(frag_type)):
    
    # Pulling local variables into new, temp variables
    output = locals()["sortOut_"+frag_type[i]]
    
    # Creating a local variable for each decoder_output
    decoder_output = "decoder_output_" + frag_type[i]
    locals()[decoder_output] = []

    for j in range(len(output)):
    




(16, 6)

### Part B: Preprocessing Decoder Data

In [16]:
#Import standard packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
import pickle
import sys

#Import metrics
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho
from Neural_Decoding.metrics import get_R2_parts

#Import decoder functions
from Neural_Decoding.decoders import KalmanFilterDecoder

In [80]:
#For the Kalman filter, we use the position, velocity, and acceleration as outputs.
#Ultimately, we are only concerned with the goodness of fit of velocity, but using them all as covariates helps performance.

for i in range(len(frag_type)):
    
    # Pulling local variables into new, temp variables
    outputX = locals()["sortX_"+frag_type[i]]
    outputY = locals()["sortY_"+frag_type[i]]
    
    # Creating a local variable for each decoder_output
    decoder_output = "decoder_output_" + frag_type[i]
    locals()[decoder_output] = []

    for j in range(len(outputX)):
        # Determine X and Y velocities 
        vel_X = (np.diff(outputX[j], axis = 0))      
        vel_X = np.concatenate((vel_X,vel_X[-1:,:]),axis=0)
        vel_Y = (np.diff(outputY[j], axis = 0))      
        vel_Y = np.concatenate((vel_Y,vel_Y[-1:,:]),axis=0)

        # Determine X and Y accelerations 
        acc_X = (np.diff(vel_X, axis = 0))   
        acc_X = np.concatenate((acc_X,acc_X[-1:,:]),axis=0)
        acc_Y = (np.diff(vel_Y, axis = 0))      
        acc_Y = np.concatenate((acc_Y,acc_Y[-1:,:]),axis=0)

        # Concatenating all 3 kinematics for X and Y components
        # Corrected decoder outputs (format = x,y vel, x,y acc, x,y pos)
        temp = np.array(np.concatenate((vel_X, vel_Y, acc_X, acc_Y, outputX[j], outputY[j]),axis=1))

        locals()[decoder_output].append(temp)

In [146]:
decoder_output_AD
decoder_output_AD[0].shape
decoder_output_AD[0][-1]

[array([[ 2.38861708e-01, -3.52387935e-01,  7.33117368e-01,
          6.09028943e-01,  2.81711322e+01,  2.71794503e+02],
        [ 9.71979076e-01,  2.56641007e-01, -2.39278454e+01,
          6.05714906e+00,  2.84099939e+01,  2.71442115e+02],
        [-2.29558664e+01,  6.31379007e+00,  2.61503313e+01,
         -6.65196604e+00,  2.93819730e+01,  2.71698756e+02],
        ...,
        [ 8.25141574e+00, -7.45860819e-01, -1.21218443e+00,
         -6.98810465e+00,  9.31909748e+00,  2.50593419e+02],
        [ 7.03923131e+00, -7.73396546e+00,  0.00000000e+00,
          0.00000000e+00,  1.75705132e+01,  2.49847558e+02],
        [ 7.03923131e+00, -7.73396546e+00,  0.00000000e+00,
          0.00000000e+00,  2.46097445e+01,  2.42113592e+02]]),
 array([[  0.92786338,   2.90068393,   1.04446409,   1.99894005,
          20.87397992, 184.40165161],
        [  1.97232747,   4.89962398,   1.49847817,   2.53531146,
          21.8018433 , 187.30233554],
        [  3.47080564,   7.43493545,   2.49662926,   

(1633, 6)

array([  7.03923131,  -7.73396546,   0.        ,   0.        ,
        24.60974452, 242.11359228])

### Part C: Partitioning and Running the Kalman Filter on the Same Buckets 

In [87]:
#Set what part of data should be part of the training/testing/validation sets

training_range=[0, 0.8]
valid_range=[0.8,0.9]
testing_range=[0.9, 1]

In [88]:
# Running decoder on X and Y components together to get the prediction nom and denom to later compute the combined XY_FVAF
from Neural_Decoding.runModelsKF import run_model_kf

for i in range(len(frag_type)):

    # Pulling local variables 
    input = locals()["sortIn_"+frag_type[i]]
    output = locals()["decoder_output_"+frag_type[i]]

    # Creating a local variable to hold (X, Y) predicted outputs for each frag type 
    parts = "pred_parts_" + frag_type[i]
    locals()[parts] = run_model_kf(input, output, training_range, testing_range, valid_range, "parts")

/Users/rbhatt6/Documents/Neural_Decoding_Dev/Neural_Decoding/runModelsKF.py:48: RuntimeWarning: invalid value encountered in true_divide
  X_train=(X_train-X_train_mean)/X_train_std
/Users/rbhatt6/Documents/Neural_Decoding_Dev/Neural_Decoding/runModelsKF.py:49: RuntimeWarning: divide by zero encountered in true_divide
  X_test=(X_test-X_train_mean)/X_train_std
/Users/rbhatt6/Documents/Neural_Decoding_Dev/Neural_Decoding/runModelsKF.py:49: RuntimeWarning: invalid value encountered in true_divide
  X_test=(X_test-X_train_mean)/X_train_std
/Users/rbhatt6/Documents/Neural_Decoding_Dev/Neural_Decoding/runModelsKF.py:50: RuntimeWarning: invalid value encountered in true_divide
  X_valid=(X_valid-X_train_mean)/X_train_std


In [94]:
pred_parts_AD[0][0][5]

105691.96777379638

In [96]:
# Compute combined XY_FVAF (velocity only)
from Neural_Decoding.metrics import compute_XY_FVAF

for i in range(len(frag_type)):

    # Pulling previously computed predicted_parts (i.e. nom and denom)
    parts = locals()[ "pred_parts_" + frag_type[i]]
    
    # Creating a local variable to hold XY_FVAFs each frag type 
    XY_FVAF = "XY_FVAF_" + frag_type[i]
    locals()[XY_FVAF] = []

    for j in range(len(parts)):
        #curr_bucket = Kalman_R2s_combined[i]
        vel_x_nom = parts[j][0][0] # dim = (curr_bucket, nom, x_vel)
        vel_x_denom = parts[j][1][0] # dim = (curr_bucket, denom, x_vel)
        vel_y_nom = parts[j][0][1] # dim = (curr_bucket, nom, y_vel)
        vel_y_denom = parts[j][1][1] # dim = (curr_bucket, denom, y_vel)

        curr_bucket_XY_FVAF = compute_XY_FVAF(vel_x_nom,vel_x_denom,vel_y_nom,vel_y_denom)
        locals()[XY_FVAF].append(curr_bucket_XY_FVAF)

In [99]:
XY_FVAF_HV

[0.1841062251340041,
 nan,
 0.15763774570315126,
 0.14912740243211653,
 0.1498301473946998,
 0.1878203662639506,
 0.15412887524048502,
 0.17529831111052663,
 0.07017912624038569,
 0.0896783076780211,
 -0.04286038172450968,
 -0.029913522789741798,
 0.04915769389920999,
 0.06663389571021072,
 0.06463894052602315,
 -0.09471064487266201]

### Part D: Partitioning and Running the Kalman Filter on Separate Training and Test Buckets